Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [89]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [90]:
pickle_file = 'data/fullyconnected/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [91]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [92]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [134]:
def add_layer(input_data, in_size, out_size, n_layer, activate_function=None):
    layer_name = 'layer%s' % n_layer
    with tf.name_scope(layer_name):
        with tf.name_scope('Weights'):
            Weights = tf.Variable(tf.truncated_normal([in_size, out_size]), name='W')
            tf.summary.histogram(layer_name + '/Weights', Weights)
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
            tf.summary.histogram(layer_name + '/biases', biases)
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.matmul(input_data, Weights) + biases
        if(activate_function is None):
            outputs = Wx_plus_b
        else:
            outputs = activate_function(Wx_plus_b)
            tf.summary.histogram(layer_name + '/outputs', outputs)
        return outputs, Weights, biases

In [135]:
batch_size = 128
image_size = 28
num_labels = 10
hidden_unit_num1 = 1024

graph = tf.Graph()
with graph.as_default():

    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    with tf.name_scope('inputs'):
        tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size), name='train_dataset')
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name='train_labels')

    hidden_layer1, W1, b1 = add_layer(tf_train_dataset, image_size*image_size, hidden_unit_num1, n_layer=1, activate_function=tf.nn.relu)
    train_output, W2, b2 = add_layer(hidden_layer1, hidden_unit_num1, num_labels, n_layer=2, activate_function=None)

    # L2 regularization, lambda = 0.02 is the best penalty rate
    l2_loss = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2)
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_output)) + 0.02 * l2_loss
    with tf.name_scope('train_optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    train_prediction = tf.nn.softmax(train_output)

    valid_hidden = tf.matmul(tf_valid_dataset, W1) + b1
    valid_output = tf.matmul(valid_hidden, W2) + b2
    valid_prediction = tf.nn.softmax(valid_output)

    test_hidden = tf.matmul(tf_test_dataset, W1) + b1
    test_output = tf.matmul(test_hidden, W2) + b2
    test_prediction = tf.nn.softmax(test_output)

### TensorBoard启动方法
---
在命令行中输入`tensorboard --logdir=目录`，其中目录为相对路径形式，且**不加引号**，生成完成后在Chrome浏览器中打开TensorBoard的启动页面（一般是`localhost:6006`）

In [136]:
num_steps = 3001

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    
    writer = tf.summary.FileWriter("logs/", sess.graph)
    merged = tf.summary.merge_all()
    
    print('Initialized...')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

        _, l, predictions = sess.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized...
Minibatch loss at step 0: 6730.554199
Minibatch accuracy: 10.2%
Validation accuracy: 20.6%
Minibatch loss at step 500: 845.142212
Minibatch accuracy: 85.2%
Validation accuracy: 60.3%
Minibatch loss at step 1000: 114.091057
Minibatch accuracy: 85.9%
Validation accuracy: 65.3%
Minibatch loss at step 1500: 15.994305
Minibatch accuracy: 85.9%
Validation accuracy: 80.9%
Minibatch loss at step 2000: 2.945396
Minibatch accuracy: 83.6%
Validation accuracy: 81.3%
Minibatch loss at step 2500: 1.141268
Minibatch accuracy: 78.9%
Validation accuracy: 82.0%
Minibatch loss at step 3000: 0.809437
Minibatch accuracy: 85.2%
Validation accuracy: 81.8%
Test accuracy: 88.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [140]:
train_dataset_small = train_dataset[:2000, :]
train_labels_small = train_labels[:2000, :]
print('Small training set', train_dataset_small.shape, train_labels_small.shape)

Small training set (2000, 784) (2000, 10)


In [141]:
batch_size = 128
image_size = 28
num_labels = 10
hidden_unit_num1 = 1024
num_steps = 3001

graph = tf.Graph()
with graph.as_default():

    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    with tf.name_scope('inputs'):
        tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size), name='train_dataset')
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name='train_labels')

    hidden_layer1, W1, b1 = add_layer(tf_train_dataset, image_size*image_size, hidden_unit_num1, n_layer=1, activate_function=tf.nn.relu)
    train_output, W2, b2 = add_layer(hidden_layer1, hidden_unit_num1, num_labels, n_layer=2, activate_function=None)

    # L2 regularization, lambda = 0.02 is the best penalty rate
    l2_loss = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2)
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_output)) + 0.02 * l2_loss
    with tf.name_scope('train_optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    train_prediction = tf.nn.softmax(train_output)

    valid_hidden = tf.matmul(tf_valid_dataset, W1) + b1
    valid_output = tf.matmul(valid_hidden, W2) + b2
    valid_prediction = tf.nn.softmax(valid_output)

    test_hidden = tf.matmul(tf_test_dataset, W1) + b1
    test_output = tf.matmul(test_hidden, W2) + b2
    test_prediction = tf.nn.softmax(test_output)

In [142]:
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    
    writer = tf.summary.FileWriter("logs/", sess.graph)
    merged = tf.summary.merge_all()
    
    print('Initialized...')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
        batch_data = train_dataset_small[offset:(offset + batch_size), :]
        batch_labels = train_labels_small[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

        _, l, predictions = sess.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized...
Minibatch loss at step 0: 6619.092773
Minibatch accuracy: 13.3%
Validation accuracy: 27.7%
Minibatch loss at step 500: 847.586243
Minibatch accuracy: 100.0%
Validation accuracy: 64.9%
Minibatch loss at step 1000: 114.591621
Minibatch accuracy: 100.0%
Validation accuracy: 69.3%
Minibatch loss at step 1500: 15.932053
Minibatch accuracy: 98.4%
Validation accuracy: 78.3%
Minibatch loss at step 2000: 2.794888
Minibatch accuracy: 95.3%
Validation accuracy: 79.7%
Minibatch loss at step 2500: 0.951144
Minibatch accuracy: 94.5%
Validation accuracy: 80.1%
Minibatch loss at step 3000: 0.599843
Minibatch accuracy: 98.4%
Validation accuracy: 80.3%
Test accuracy: 86.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
